#  Evaluation Demo

### Imports

In [37]:
import io
import base64
import glob
import os

import gzip
import json
import numpy as np
import torch
from IPython.display import HTML
from skvideo.io import FFmpegReader, ffprobe, vwrite
from torch.autograd import Variable
from ptcap.trainers import DataParallelWrapper
from ptcap.scores import ( caption_accuracy, first_token_accuracy, token_accuracy)
from ptcap.data.annotation_parser import JsonParser
from collections import OrderedDict
from collections import Counter, namedtuple

In [38]:
int2label = {
     0: 'Approaching [something] with your camera',
     1: 'Attaching [something] to [something]',
     2: 'Bending [something] so that it deforms',
     3: 'Bending [something] until it breaks',
     4: 'Burying [something] in [something]',
     5: 'Closing [something]',
     6: 'Covering [something] with [something]',
     7: 'Digging [something] out of [something]',
     8: 'Dropping [something] behind [something]',
     9: 'Dropping [something] in front of [something]',
     10: 'Dropping [something] into [something]',
     11: 'Dropping [something] next to [something]',
     12: 'Dropping [something] onto [something]',
     13: 'Failing to put [something] into [something] because [something] does not fit',
     14: 'Folding [something]',
     15: 'Hitting [something] with [something]',
     16: 'Holding [something]',
     17: 'Holding [something] behind [something]',
     18: 'Holding [something] in front of [something]',
     19: 'Holding [something] next to [something]',
     20: 'Holding [something] over [something]',
     21: 'Laying [something] on the table on its side, not upright',
     22: 'Letting [something] roll along a flat surface',
     23: 'Letting [something] roll down a slanted surface',
     24: 'Letting [something] roll up a slanted surface, so it rolls back down',
     25: 'Lifting [something] up completely without letting it drop down',
     26: 'Lifting [something] up completely, then letting it drop down',
     27: 'Lifting [something] with [something] on it',
     28: 'Lifting a surface with [something] on it but not enough for it to slide down',
     29: 'Lifting a surface with [something] on it until it starts sliding down',
     30: 'Lifting up one end of [something] without letting it drop down',
     31: 'Lifting up one end of [something], then letting it drop down',
     32: 'Moving [part] of [something]',
     33: 'Moving [something] across a surface until it falls down',
     34: 'Moving [something] across a surface without it falling down',
     35: 'Moving [something] and [something] away from each other',
     36: 'Moving [something] and [something] closer to each other',
     37: 'Moving [something] and [something] so they collide with each other',
     38: 'Moving [something] and [something] so they pass each other',
     39: 'Moving [something] away from [something]',
     40: 'Moving [something] away from the camera',
     41: 'Moving [something] closer to [something]',
     42: 'Moving [something] down',
     43: 'Moving [something] towards the camera',
     44: 'Moving [something] up',
     45: 'Moving away from [something] with your camera',
     46: 'Opening [something]',
     47: 'Picking [something] up',
     48: 'Piling [something] up',
     49: 'Plugging [something] into [something]',
     50: 'Plugging [something] into [something] but pulling it right out as you remove your hand',
     51: 'Poking [something] so it slightly moves',
     52: 'Poking [something] so lightly that it doesn’t or almost doesn’t move',
     53: 'Poking [something] so that it falls over',
     54: 'Poking [something] so that it spins around',
     55: 'Poking a hole into [some substance]',
     56: 'Poking a hole into [something soft]',
     57: 'Poking a stack of [something] so the stack collapses',
     58: 'Poking a stack of [something] without the stack collapsing',
     59: 'Pouring [something] into [something]',
     60: 'Pouring [something] into [something] until it overflows',
     61: 'Pouring [something] onto [something]',
     62: 'Pouring [something] out of [something]',
     63: 'Pretending or failing to wipe [something] off of [something]',
     64: 'Pretending or trying and failing to twist [something]',
     65: 'Pretending to be tearing [something that is not tearable]',
     66: 'Pretending to close [something] without actually closing it',
     67: 'Pretending to open [something] without actually opening it',
     68: 'Pretending to pick [something] up',
     69: 'Pretending to poke [something]',
     70: 'Pretending to pour [something] out of [something], but [something] is empty',
     71: 'Pretending to put [something] behind [something]',
     72: 'Pretending to put [something] into [something]',
     73: 'Pretending to put [something] next to [something]',
     74: 'Pretending to put [something] on a surface',
     75: 'Pretending to put [something] onto [something]',
     76: 'Pretending to put [something] underneath [something]',
     77: 'Pretending to scoop [something] up with [something]',
     78: 'Pretending to spread “air” onto [something]',
     79: 'Pretending to sprinkle "air" onto [something]',
     80: 'Pretending to squeeze [something]',
     81: 'Pretending to take [something] from [somewhere]',
     82: 'Pretending to take [something] out of [something]',
     83: 'Pretending to throw [something]',
     84: 'Pretending to turn [something] upside down',
     85: 'Pulling [something] from behind of [something]',
     86: 'Pulling [something] from left to right',
     87: 'Pulling [something] from right to left',
     88: 'Pulling [something] onto [something]',
     89: 'Pulling [something] out of [something]',
     90: 'Pulling two ends of [something] but nothing happens',
     91: 'Pulling two ends of [something] so that it gets stretched',
     92: 'Pulling two ends of [something] so that it separates into two pieces',
     93: 'Pushing [something] from left to right',
     94: 'Pushing [something] from right to left',
     95: 'Pushing [something] off of [something]',
     96: 'Pushing [something] onto [something]',
     97: 'Pushing [something] so it spins',
     98: "Pushing [something] so that it almost falls off but doesn't",
     99: 'Pushing [something] so that it falls off the table',
     100: 'Pushing [something] so that it slightly moves',
     101: 'Pushing [something] with [something]',
     102: 'Putting [number of] [something] onto [something]',
     103: 'Putting [something similar to other things that are already on the table]',
     104: 'Putting [something that cannot actually stand upright] upright on the table, so it falls on its side',
     105: 'Putting [something] and [something] on the table',
     106: 'Putting [something] behind [something]',
     107: 'Putting [something] in front of [something]',
     108: 'Putting [something] into [something]',
     109: 'Putting [something] next to [something]',
     110: 'Putting [something] on a flat surface without letting it roll',
     111: 'Putting [something] on a surface',
     112: 'Putting [something] on the edge of [something] so it is not supported and falls down',
     113: 'Putting [something] onto [something else that cannot support it] so it falls down',
     114: 'Putting [something] onto [something]',
     115: 'Putting [something] onto a slanted surface but it doesn’t glide down',
     116: 'Putting [something] that can’t roll onto a slanted surface, so it slides down',
     117: 'Putting [something] that can’t roll onto a slanted surface, so it stays where it is',
     118: 'Putting [something] underneath [something]',
     119: 'Putting [something] upright on the table',
     120: 'Putting [something], [something] and [something] on the table',
     121: 'Removing [something], revealing [something] behind',
     122: 'Rolling [something] on a flat surface',
     123: 'Scooping [something] up with [something]',
     124: 'Show a shadow of [something] that is moving. ',
     125: 'Show a shadow of [something], making sure the shadow is not moving. ',
     126: 'Showing [something] behind [something]',
     127: 'Showing [something] next to [something]',
     128: 'Showing [something] on top of [something]',
     129: 'Showing [something] to the camera',
     130: 'Showing a photo of [something] to the camera',
     131: 'Showing a shadow of [something] that is moving.',
     132: 'Showing a shadow of [something], making sure the shadow is not moving.',
     133: 'Showing that [something] is empty',
     134: 'Showing that [something] is inside [something]',
     135: 'Spilling [something] behind [something]',
     136: 'Spilling [something] next to [something]',
     137: 'Spilling [something] onto [something]',
     138: 'Spinning [something] so it continues spinning',
     139: 'Spinning [something] that quickly stops spinning',
     140: 'Spreading [something] onto [something]',
     141: 'Sprinkling [something] onto [something]',
     142: 'Squeezing [something]',
     143: 'Stacking [number of] [something]',
     144: 'Stuffing [something] into [something]',
     145: 'Taking [one of many similar things on the table]',
     146: 'Taking [something] from [somewhere]',
     147: 'Taking [something] out of [something]',
     148: 'Tearing [something] into two pieces',
     149: 'Tearing [something] just a little bit',
     150: 'Throwing [something]',
     151: 'Throwing [something] against [something]',
     152: 'Throwing [something] in the air and catching it',
     153: 'Throwing [something] in the air and letting it fall',
     154: 'Throwing [something] onto a surface',
     155: 'Tilting [something] with [something] on it slightly so it doesn’t fall down',
     156: 'Tilting [something] with [something] on it until it falls off',
     157: 'Tipping [something] over',
     158: 'Tipping [something] with [something in it] over, so [something in it] falls out',
     159: 'Touching (without moving) [part] of [something]',
     160: 'Trying but failing to attach [something] to [something] because it doesn’t stick',
     161: 'Trying to bend [something unbendable] so nothing happens',
     162: 'Trying to pour [something] into [something], but missing so it spills next to it',
     163: 'Turning [something] upside down',
     164: 'Turning the camera downwards while filming [something]',
     165: 'Turning the camera left while filming [something]',
     166: 'Turning the camera right while filming [something]',
     167: 'Turning the camera upwards while filming [something]',
     168: 'Twisting (wringing) [something] wet until water comes out',
     169: 'Twisting [something]',
     170: 'Uncovering [something]',
     171: 'Unfolding [something]',
     172: 'Wiping [something] off of [something]',
     173: '[Something] being deflected from [something]',
     174: '[Something] colliding with [something] and both are being deflected',
     175: '[Something] colliding with [something] and both come to a halt',
     176: '[Something] falling like a feather or paper',
     177: '[Something] falling like a rock'
}

In [39]:
stop_words = ['A', 'AN', 'THE', '<END>']

def safe_div(x,y):
    if y == 0:
        return 0
    return x / y
def fscore(precision, recall, beta=1):
    numerator = (1.0 + (beta ** 2)) * precision * recall
    denominator = ((beta ** 2) * precision) + recall
    return {"fscore": safe_div(numerator, denominator)}
class LCS(object):
    """
    The main functionality of this class is to compute the LCS (Lowest Common
    Subsequence) between a caption and prediction. By default, it returns the
    precision and recall values calculated based on the LCS between a prediction
    and a caption.
    """
    def __init__(self, functions_list, tokenizer):
        """
        Initializes functions_list and tokenizer.
        Args:
        functions_list: A list of the functions that will be applied on the
        precision and recall values calculated based on the LCS between a
        prediction and a caption.
        """

        self.functions_list = functions_list
        self.scores_container = OrderedDict()
        self.scores_dict = OrderedDict()
        self.tokenizer = tokenizer

    def __call__(self, outputs):
        string_predictions = [self.tokenizer.get_string(str_pred.data.numpy())
                              for str_pred in outputs.predictions]
        return self.score_batch(string_predictions, outputs.string_captions)

    def collect_scores(self, batch_scores_dict, scores_dict):
        for metric, metric_value in scores_dict.items():
            if metric not in batch_scores_dict:
                batch_scores_dict[metric] = [metric_value]
            else:
                batch_scores_dict[metric].append(metric_value)
        return batch_scores_dict

    @classmethod
    def compute_lcs(cls, prediction, caption):
        num_rows = len(prediction)
        num_cols = len(caption)

        table = [[0] * (num_cols + 1) for _ in range(num_rows + 1)]
        for i in range(1, num_rows + 1):
            for j in range(1, num_cols + 1):
                if prediction[i - 1] == caption[j - 1]:
                    table[i][j] = table[i - 1][j - 1] + 1
                else:
                    table[i][j] = max(table[i][j - 1], table[i - 1][j])
        return table, table[num_rows][num_cols]

    def mean_scores(self, batch_scores_dict):
        for metric, metric_value in batch_scores_dict.items():
            batch_scores_dict[metric] = np.mean(metric_value)
        return batch_scores_dict

    def score_batch(self, predictions, captions):
        assert len(predictions) == len(captions)

        batch_scores_dict = OrderedDict()
        for count, (prediction, caption) in enumerate(zip(predictions,
                                                          captions)):
            scores_dict = self.score_sample(prediction.split(), caption.split())
            batch_scores_dict = self.collect_scores(batch_scores_dict,
                                                    scores_dict)

        batch_scores_dict = self.mean_scores(batch_scores_dict)
        return batch_scores_dict

    def score_sample(self, prediction, caption):
        scores_dict = OrderedDict()
        _, lcs_score = self.compute_lcs(prediction, caption)
        scores_dict["precision"] = safe_div(lcs_score, len(prediction))
        scores_dict["recall"] = safe_div(lcs_score, len(caption))

        for score_function in self.functions_list:
            scores_dict.update(score_function(scores_dict["precision"],
                                              scores_dict["recall"]))

        return scores_dict

### Tool to deal with mpeg videos

In [40]:
def show_video(video_filenames):
    """
    Tool to display videos inside the notebook.
    """
    
    if type(video_filenames) is not list:
        video_filenames = [video_filenames]
    
    html_code = ''
    for filename in video_filenames:
        video = io.open(filename, 'r+b').read()
        encoded = base64.b64encode(video)
        html_code += '''
        <video alt="test" width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        '''.format(encoded.decode('ascii'))
        
    return HTML(data= html_code)


def open_mpeg_video(fname, framerate, size):
    """
    Open an mpeg video, and return it as a numpy array.
    """
    
    metadata = ffprobe(fname)
    duration = float(metadata['video']['@duration'])
    # Compute corresponding nb of frames
    nframes = int(duration * framerate)
    oargs = {
        "-r": "%d" % framerate,
        "-vframes": "%d" % nframes,
        "-s": "%dx%d" % (size[0], size[1])
    }
    # Open file
    reader = FFmpegReader(fname, inputdict={}, outputdict=oargs)
    video = []
    # Get frames until there is no more
    for frame in reader.nextFrame():
        video.append(frame)
    # Return as a numpy array
    return np.array(video)

## A) Model Instantiation

In [25]:
from ptcap.model.captioners import EncoderDecoder
from ptcap.model.external_encoders import FCEncoder, JesterEncoder, BIJesterEncoder
from ptcap.model.decoders import LSTMDecoder, CoupledLSTMDecoder
  
#net = FullyConvolutionalNet(num_classes=178)jester1024_cutoff_300_ssssssss/

net = EncoderDecoder(
        encoder=BIJesterEncoder,
        decoder=CoupledLSTMDecoder,
        encoder_kwargs={"freeze": False},#, "pretrained_path": "/home/farzaneh/PycharmProjects/pretrained_nets/fully_conv_net_on_smtsmt_20170627/model.checkpoint"},
        decoder_kwargs={"embedding_size": 256, "hidden_size": 512, "num_lstm_layers": 2, 
        "vocab_size": 2578, "num_step" :17}, 
        gpus=[0]).cuda()
net = DataParallelWrapper(net, device_ids=[0]).cuda(0)

gpus: [0]


## B) Load weights

In [26]:
path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/v2_gulp160_labels_classif0.1_cap0.9'
# path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/clapnet_captioning_only_f0.1'

checkpoint = torch.load(path + '/model.best')


net.load_state_dict(checkpoint["model"])


## C) Load Tokenizer

In [27]:
from ptcap.data.tokenizer import Tokenizer

USER_MAXLEN=17
tokenizer = Tokenizer(user_maxlen=USER_MAXLEN)
tokenizer.load_dictionaries(path)

## D) Testing 

In [28]:
TMP_VIDEO_FILENAME = 'tmp.mp4'

def unpreprocess(video):
    video = video.data.numpy()[0]
    video = 64. * video.transpose(1, 2, 3, 0)
    return np.array(video, 'uint8')

def demo(net, preprocessor, filename, top_n=5):
    # Open mpeg file and get a numpy array
    video_uint8 = open_mpeg_video(filename, 12, [128, 128])
    # Preprocessing
    video = preprocessor(video_uint8)
    # Convert to torch variable
    video = Variable(torch.from_numpy(video[None]), volatile=True).cuda()
    empty_caption = Variable(torch.zeros([1, 1]), volatile=True).long().cuda()
    
    # Compute predictions
    pred, class_pred = net.forward((video, empty_caption), use_teacher_forcing=False)
    # Convert to numpy 
    pred = np.exp(pred.cpu().data.numpy())[0]
        
    pred_argmax = np.argmax(pred, axis=1)
    decoded_pred = tokenizer.decode_caption(pred_argmax)
    beautiful_caption = " ".join(str(e+" ") for e in decoded_pred if "<END>" not in e)
    #print('__CAPTION__: {}'.format(beautiful_caption))
    
    
    # Class index
    class_index = torch.max(class_pred, dim=1)[1].cpu().data[0]
    cls = int2label[class_index]
    #print('ACTION: {:60s}\n'.format(cls))
    
    
    matched_action = get_template(decoded_pred, templates, tokenizer)
    # print(actions)
    objects = get_object_tokens(decoded_pred, matched_action[0][0])

    
    objects_list = extract_objects(objects)
    # Print class name with proba
    # Save input video in tmp file
    vwrite(TMP_VIDEO_FILENAME, unpreprocess(video.cpu()))
    return beautiful_caption, cls, objects_list, matched_action[0][0]


def path_generator(annotation_path, root_path):
    with gzip.open(annotation_path, "rt") as f:
        annotations = json.load(f)
    files = [elem['file'] for elem in annotations]
    labels = [elem['label'] for elem in annotations]
    placeholders = [elem['placeholders'] for elem in annotations] 
    actions = [elem['template'].replace("[","").replace("]", "").replace(",","").upper() for elem in annotations]
    
    return ((os.path.join(root_path, f), label, a, p) for f,label,p, a in zip(files, labels, placeholders, actions))

In [29]:
# Path generator
path_gen = path_generator('/data/20bn-somethingsomething/json/test_20170929.json.gz', 
                          '/data/20bn-somethingsomething/videos')
# Put the netwoark in evaluation mode
_ = net.eval()

#### Preprocessor

In [41]:
from rtorchn.data.preprocessing import default_evaluation_preprocesser

preprocessor = default_evaluation_preprocesser([48, 128, 128], 64.)

In [31]:
for i in range(5000):
    path_to_video, label, _,_ = next(path_gen)

## Longest Common Subsequence

In [32]:
def get_template(candidates, templates, tokenizer):
   
    lcs = LCS([fscore], tokenizer)
    
    max_templates = []
    #print("There are {} templates".format(len(templates)))

    candidates = [" ".join(candidates)]
    for candidate in candidates:
        
        max_lcs_template = ""
        max_lcs_value = -1
        for template in templates:
            lcs_value = compute_LCS(lcs, candidate, template, tokenizer)
            if lcs_value > max_lcs_value:
                max_lcs_template = template
                max_lcs_value = lcs_value
        max_templates.append((max_lcs_template, max_lcs_value))
#         print("Candidate: {}".format(candidate))
        #print("MATCHED ACTION : {}".format(max_lcs_template))

    return max_templates


def compute_LCS(lcs, candidate, template, tokenizer):
    encoded_caption = Variable(
        torch.LongTensor([tokenizer.encode_caption(candidate)]))
    encoded_prediction = Variable(
        torch.LongTensor([tokenizer.encode_caption(template)]))
    score_attr = namedtuple("ScoresAttr", "string_captions captions predictions")
    in_tuple = score_attr([candidate], encoded_caption, encoded_prediction)
    lcs_output = lcs(in_tuple)
    return lcs_output['fscore']

def extract_objects(object_tokens_list):
    
    objects_list = []
    if len(object_tokens_list) == 0:
        return objects_list
    
    next_token_ind =  object_tokens_list[0][0]
    current_object = ""
    for  (ind, token) in object_tokens_list:
        if  next_token_ind == ind:
            current_object += token+" "
        else:
            objects_list.append(current_object+" ")
            current_object = token
            next_token_ind = ind
        next_token_ind += 1
        
        
    if len(current_object)>0:
        objects_list.append(current_object)
               
    #print("PREDICTED OBJECTS: {}".format(objects_list))
    return objects_list



def get_object_tokens(caption, template):
    return [(i,token) for (i,token) in enumerate(caption) if token not in template and token not in stop_words]

In [33]:
articles= ["the", "a", "an", "A", "An", "The"]

annotations = JsonParser.open_annotation("/data/20bn-somethingsomething/json/train_20171031.json.gz")
templates = np.unique(annotations["template"]) # A list of templates
objects = annotations["placeholders"]
obj_tokens = [token for token in objects if token not in stop_words]
all_obj=[item for sublist in objects for item in sublist]
filtered_obj =  [" ".join(obj) for obj in all_obj for  token in obj if token not in articles]
templates = [" ".join(tokenizer.tokenize(t)) for t in templates]


# get_objects(sentences1[0].split(), sentence1_templates[0].split())



In [34]:
articles= ["the", "a", "an", "A", "An", "The"]

fil3 = [token.upper() for obj in all_obj for token in obj.split(" ")]
fil2 = [[token for token in obj.split(" ") if token not in articles ] for obj in all_obj]
fil = list( map(lambda p:" ".join([token for token in p.split(" ") if token not in articles ]), all_obj))



In [35]:
all_objects = {}
correct_objects = {}
all_actions = {}
correct_actions = {}

for i in range(1000):
    path_to_video, target_caption, target_action, target_objects = next(path_gen)
    for p in target_objects:
        p_tokens = p.split(" ")
        for pto in p_tokens:
            all_objects[pto.strip().upper()] = all_objects.get(pto.strip().upper(), 0) + 1
        
    all_actions[target_action] = all_actions.get(target_action, 0) + 1

   
    print('sample {}'.format(i)) 
    pred_caption, pred_action, pred_objects, matched_action = demo(net, preprocessor, path_to_video)
    for (i,o) in enumerate(pred_objects):
        o_tokens = o.strip().split(" ")
        for oto in o_tokens:
            if i<len(target_objects) and oto in target_objects[i].upper():
                correct_objects[oto] = correct_objects.get(oto, 0) + 1
                print("woohoo")
               
          
    print('TARGET CAPTION: {}'.format(target_caption))
    print('PRED   CAPTION: {}\n'.format(pred_caption))
    print('TARGET  ACTION: {}'.format(target_action))
    print('CLASSIF ACTION: {}'.format(pred_action))
    print('CAPTION ACTION: {}\n'.format(matched_action))
            
    print('TARGET OBJECTS: {}'.format(target_objects))
    print('PRED   OBJECTS: {}\n'.format(pred_objects))


    if matched_action == target_action:
        print("yesss")
        correct_actions[matched_action] = correct_actions.get(matched_action, 0) + 1
        
    print('{}\n'.format('-'*65))
    
print(all_objects)
print(correct_objects)

print(all_actions)
print(correct_actions)
    

sample 0
TARGET CAPTION: moving ball away from comp
PRED   CAPTION: MOVING  AWAY  FROM  A  <UNK>  WITH  YOUR  CAMERA 

TARGET  ACTION: MOVING SOMETHING AWAY FROM SOMETHING
CLASSIF ACTION: Moving [something] away from [something]
CAPTION ACTION: MOVING AWAY FROM SOMETHING WITH YOUR CAMERA

TARGET OBJECTS: ['ball', 'comp']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 1
TARGET CAPTION: holding cards
PRED   CAPTION: SHOWING  A  PHOTO  OF  <UNK>  TO  THE  CAMERA 

TARGET  ACTION: HOLDING SOMETHING
CLASSIF ACTION: Show a shadow of [something], making sure the shadow is not moving. 
CAPTION ACTION: SHOWING A PHOTO OF SOMETHING TO THE CAMERA

TARGET OBJECTS: ['cards']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 2
TARGET CAPTION: holding remote next to doll
PRED   CAPTION: SHOWING  A  PHOTO  OF  <UNK>  TO  THE  CAMERA 

TARGET  ACTION: HOLDING SOMETHING NEXT TO SOMETHING
CLASSIF AC

TARGET CAPTION: pretending to spread “air” onto Kitchen table
PRED   CAPTION: PUSHING  BOTTLE  FROM  RIGHT  TO  LEFT 

TARGET  ACTION: PRETENDING TO SPREAD “AIR” ONTO SOMETHING
CLASSIF ACTION: Pushing [something] from right to left
CAPTION ACTION: PUSHING SOMETHING FROM RIGHT TO LEFT

TARGET OBJECTS: ['Kitchen table']
PRED   OBJECTS: ['BOTTLE ']

-----------------------------------------------------------------

sample 21
TARGET CAPTION: putting a wallet  next to a glass
PRED   CAPTION: PUTTING  SOMETHING  SIMILAR  TO  OTHER  THINGS  THAT  ARE  ALREADY  ON  THE  <UNK> 

TARGET  ACTION: PUTTING SOMETHING NEXT TO SOMETHING
CLASSIF ACTION: Putting [something] on a surface
CAPTION ACTION: PUTTING SOMETHING SIMILAR TO OTHER THINGS THAT ARE ALREADY ON THE TABLE

TARGET OBJECTS: ['a wallet ', 'a glass']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 22
TARGET CAPTION: pretending to put a computer chip into a glass
PRED   CAPTION: POKING  

TARGET CAPTION: putting Keys into Bag
PRED   CAPTION: PRETENDING  TO  PUT  THE  TABLE  ONTO  SOMETHING 

TARGET  ACTION: PUTTING SOMETHING INTO SOMETHING
CLASSIF ACTION: Pretending to put [something] behind [something]
CAPTION ACTION: PRETENDING TO PUT SOMETHING ONTO SOMETHING

TARGET OBJECTS: ['Keys', 'Bag']
PRED   OBJECTS: ['TABLE ']

-----------------------------------------------------------------

sample 40
TARGET CAPTION: putting a pair of spectacles onto a dashboard
PRED   CAPTION: PUTTING  A  <UNK>  ON  A  SURFACE 

TARGET  ACTION: PUTTING SOMETHING ONTO SOMETHING
CLASSIF ACTION: Putting [something] on a surface
CAPTION ACTION: PUTTING SOMETHING ON A SURFACE

TARGET OBJECTS: ['a pair of spectacles', 'a dashboard']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 41
TARGET CAPTION: taking a jar out of a shelf
PRED   CAPTION: UNCOVERING  PENCIL 

TARGET  ACTION: TAKING SOMETHING OUT OF SOMETHING
CLASSIF ACTION: Turning the came

TARGET CAPTION: showing a toy pig next to a toy cat
PRED   CAPTION: SHOWING  A  PHOTO  OF  <UNK>  TO  THE  CAMERA 

TARGET  ACTION: SHOWING SOMETHING NEXT TO SOMETHING
CLASSIF ACTION: Showing [something] on top of [something]
CAPTION ACTION: SHOWING A PHOTO OF SOMETHING TO THE CAMERA

TARGET OBJECTS: ['a toy pig', 'a toy cat']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 60
TARGET CAPTION: turning Wood box upside down
PRED   CAPTION: TURNING  A  REMOTE  UPSIDE  DOWN 

TARGET  ACTION: TURNING SOMETHING UPSIDE DOWN
CLASSIF ACTION: Touching (without moving) [part] of [something]
CAPTION ACTION: TURNING SOMETHING UPSIDE DOWN

TARGET OBJECTS: ['Wood box']
PRED   OBJECTS: ['REMOTE ']

yesss
-----------------------------------------------------------------

sample 61
TARGET CAPTION: pouring wine into a wine glass
PRED   CAPTION: POURING  A  BOTTLE  OUT  OF  SOMETHING 

TARGET  ACTION: POURING SOMETHING INTO SOMETHING
CLASSIF ACTION: Pou

TARGET CAPTION: putting bottle on the edge of table so it is not supported and falls down
PRED   CAPTION: PRETENDING  TO  PICK  A  <UNK>  UP 

TARGET  ACTION: PUTTING SOMETHING ON THE EDGE OF SOMETHING SO IT IS NOT SUPPORTED AND FALLS DOWN
CLASSIF ACTION: Putting [something similar to other things that are already on the table]
CAPTION ACTION: PRETENDING TO PICK SOMETHING UP

TARGET OBJECTS: ['bottle', 'table']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 80
TARGET CAPTION: pushing cup so that it falls off the table
PRED   CAPTION: PUSHING  A  BOTTLE  SO  THAT  IT  FALLS  OFF  THE  TABLE 

TARGET  ACTION: PUSHING SOMETHING SO THAT IT FALLS OFF THE TABLE
CLASSIF ACTION: Pushing [something] so that it falls off the table
CAPTION ACTION: PUSHING SOMETHING SO THAT IT FALLS OFF THE TABLE

TARGET OBJECTS: ['cup']
PRED   OBJECTS: ['BOTTLE ']

yesss
-----------------------------------------------------------------

sample 81
TARGET CAPTI

...

TARGET CAPTION: tearing construction paper into two pieces
PRED   CAPTION: SHOWING  A  PHOTO  OF  <UNK>  TO  THE  CAMERA 

TARGET  ACTION: TEARING SOMETHING INTO TWO PIECES
CLASSIF ACTION: Showing [something] on top of [something]
CAPTION ACTION: SHOWING A PHOTO OF SOMETHING TO THE CAMERA

TARGET OBJECTS: ['construction paper']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 914
TARGET CAPTION: throwing a pair of pliers against the floor
PRED   CAPTION: SOMETHING  FALLING  LIKE  A  <UNK>  FALLING  LIKE  A  ROCK 

TARGET  ACTION: THROWING SOMETHING AGAINST SOMETHING
CLASSIF ACTION: [Something] being deflected from [something]
CAPTION ACTION: SOMETHING FALLING LIKE A ROCK

TARGET OBJECTS: ['a pair of pliers', 'the floor']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 915
TARGET CAPTION: pouring ivi onto glass
PRED   CAPTION: POURING  GLASS  INTO  SOMETHING 

TARGET  ACTION: POURI

TARGET CAPTION: covering cup with cloth
PRED   CAPTION: COVERING  A  BLANKET  WITH  SOMETHING 

TARGET  ACTION: COVERING SOMETHING WITH SOMETHING
CLASSIF ACTION: Covering [something] with [something]
CAPTION ACTION: COVERING SOMETHING WITH SOMETHING

TARGET OBJECTS: ['cup', 'cloth']
PRED   OBJECTS: ['BLANKET ']

yesss
-----------------------------------------------------------------

sample 933
TARGET CAPTION: pulling Toy car out of Jar
PRED   CAPTION: TAKING  TABLE  FROM  SOMEWHERE 

TARGET  ACTION: PULLING SOMETHING OUT OF SOMETHING
CLASSIF ACTION: Squeezing [something]
CAPTION ACTION: TAKING SOMETHING FROM SOMEWHERE

TARGET OBJECTS: ['Toy car', 'Jar']
PRED   OBJECTS: ['TABLE ']

-----------------------------------------------------------------

sample 934
TARGET CAPTION: dropping pen onto table
PRED   CAPTION: SOMETHING  FALLING  LIKE  A  FEATHER  OR  <UNK>  FALLING  LIKE  A  FEATHER  OR 

TARGET  ACTION: DROPPING SOMETHING ONTO SOMETHING
CLASSIF ACTION: [Something] being deflected 

TARGET CAPTION: moving glass closer to mug
PRED   CAPTION: MOVING  A  BOTTLE  AND  SOMETHING  CLOSER  TO  EACH  OTHER 

TARGET  ACTION: MOVING SOMETHING CLOSER TO SOMETHING
CLASSIF ACTION: Moving [something] and [something] closer to each other
CAPTION ACTION: MOVING SOMETHING AND SOMETHING CLOSER TO EACH OTHER

TARGET OBJECTS: ['glass', 'mug']
PRED   OBJECTS: ['BOTTLE ']

-----------------------------------------------------------------

sample 953
TARGET CAPTION: throwing watch
PRED   CAPTION: THROWING  WALL  AGAINST  SOMETHING 

TARGET  ACTION: THROWING SOMETHING
CLASSIF ACTION: Taking [something] from [somewhere]
CAPTION ACTION: THROWING SOMETHING AGAINST SOMETHING

TARGET OBJECTS: ['watch']
PRED   OBJECTS: ['WALL ']

-----------------------------------------------------------------

sample 954
TARGET CAPTION: folding Newspaper 
PRED   CAPTION: THROWING  A  ROCK  IN  THE  AIR  AND  LETTING  IT  FALL 

TARGET  ACTION: FOLDING SOMETHING
CLASSIF ACTION: Holding [something] in front of

TARGET CAPTION: plugging charger into tablet
PRED   CAPTION: PLUGGING  LAPTOP  INTO  SOMETHING 

TARGET  ACTION: PLUGGING SOMETHING INTO SOMETHING
CLASSIF ACTION: Plugging [something] into [something] but pulling it right out as you remove your hand
CAPTION ACTION: PLUGGING SOMETHING INTO SOMETHING

TARGET OBJECTS: ['charger', 'tablet']
PRED   OBJECTS: ['LAPTOP ']

yesss
-----------------------------------------------------------------

sample 972
TARGET CAPTION: taking make-up out of toiletry bag
PRED   CAPTION: UNCOVERING  PLATE 

TARGET  ACTION: TAKING SOMETHING OUT OF SOMETHING
CLASSIF ACTION: Turning the camera right while filming [something]
CAPTION ACTION: UNCOVERING SOMETHING

TARGET OBJECTS: ['make-up', 'toiletry bag']
PRED   OBJECTS: ['PLATE ']

-----------------------------------------------------------------

sample 973
TARGET CAPTION: putting water bottle upright on the table
PRED   CAPTION: THROWING  BED  AGAINST  SOMETHING 

TARGET  ACTION: PUTTING SOMETHING UPRIGHT ON T

TARGET CAPTION: dropping Q-tips into A coffee can
PRED   CAPTION: SHOWING  THAT  A  BOWL  IS  INSIDE  SOMETHING 

TARGET  ACTION: DROPPING SOMETHING INTO SOMETHING
CLASSIF ACTION: Showing [something] to the camera
CAPTION ACTION: SHOWING THAT SOMETHING IS INSIDE SOMETHING

TARGET OBJECTS: ['Q-tips', 'A coffee can']
PRED   OBJECTS: ['BOWL ']

-----------------------------------------------------------------

sample 992
TARGET CAPTION: moving page of book
PRED   CAPTION: PUTTING  SOMETHING  SIMILAR  TO  OTHER  THINGS  THAT  ARE  ALREADY  ON  THE  <UNK> 

TARGET  ACTION: MOVING PART OF SOMETHING
CLASSIF ACTION: Putting [something] and [something] on the table
CAPTION ACTION: PUTTING SOMETHING SIMILAR TO OTHER THINGS THAT ARE ALREADY ON THE TABLE

TARGET OBJECTS: ['page', 'book']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

sample 993
TARGET CAPTION: showing that a cup is empty
PRED   CAPTION: SHOWING  THAT  GLASS  IS  EMPTY 

TARGET  ACTIO

In [36]:
for ca in correct_actions:
    
    print("{}/{} of {} actions correct".format(correct_actions[ca], all_actions[ca], ca))

2/2 of TURNING THE CAMERA UPWARDS WHILE FILMING SOMETHING actions correct
1/4 of TILTING SOMETHING WITH SOMETHING ON IT UNTIL IT FALLS OFF actions correct
4/11 of POURING SOMETHING INTO SOMETHING actions correct
1/3 of SCOOPING SOMETHING UP WITH SOMETHING actions correct
3/9 of MOVING SOMETHING CLOSER TO SOMETHING actions correct
1/5 of LIFTING A SURFACE WITH SOMETHING ON IT UNTIL IT STARTS SLIDING DOWN actions correct
1/2 of PUTTING SOMETHING UNDERNEATH SOMETHING actions correct
3/8 of SPINNING SOMETHING THAT QUICKLY STOPS SPINNING actions correct
2/6 of PILING SOMETHING UP actions correct
1/11 of PUTTING SOMETHING INTO SOMETHING actions correct
1/1 of LIFTING A SURFACE WITH SOMETHING ON IT BUT NOT ENOUGH FOR IT TO SLIDE DOWN actions correct
1/5 of MOVING SOMETHING ACROSS A SURFACE UNTIL IT FALLS DOWN actions correct
1/1 of PRETENDING TO PUT SOMETHING ON A SURFACE actions correct
1/11 of TAKING SOMETHING OUT OF SOMETHING actions correct
2/8 of PUSHING SOMETHING SO THAT IT SLIGHTLY MOV

In [42]:
for correct_key in correct_objects.keys():
    denom = 0
    if correct_key in all_objects.keys():
        denom += all_objects[correct_key]
        #print("{}:{}".format(correct_key, all_objects[correct_key]))
    #for j in all_objects.keys():
    #    if correct_key in j or correct_key in j:
    #        print("{}:{}".format(j, all_objects[j]))
    #        denom += all_objects[j]
        
                    
    print (">>model got  {}/{} of '{}'s correct".format(correct_objects[correct_key],denom, correct_key ))
    print("-"*100)
    

>>model got  1/4 of 'COMB's correct
----------------------------------------------------------------------------------------------------
>>model got  2/63 of 'BOX's correct
----------------------------------------------------------------------------------------------------
>>model got  1/8 of 'PILLOW's correct
----------------------------------------------------------------------------------------------------
>>model got  1/5 of 'SPINNER's correct
----------------------------------------------------------------------------------------------------
>>model got  19/74 of 'PAPER's correct
----------------------------------------------------------------------------------------------------
>>model got  1/1 of 'SPANNER's correct
----------------------------------------------------------------------------------------------------
>>model got  1/3 of 'DRAWER's correct
----------------------------------------------------------------------------------------------------
>>model got  1/32 of 'GLASS'

In [43]:
a = sum(correct_objects.values())
b = sum(all_objects.values())

print("{} out of {} objects are correctly predicted: {:.2}% ".format(a, b, a/b*100))

63 out of 2386 objects are correctly predicted: 2.6% 


In [44]:
len(all_objects.keys())

599

In [45]:
c = sum(correct_actions.values())
d = sum(all_actions.values())

print("{} out of {} actions are correctly predicted: {:}% ".format(c, d, c/d*100))

279 out of 1000 actions are correctly predicted: 27.900000000000002% 


In [46]:
len(all_actions.keys())

164